In [1]:
!pip install transformers accelerate einops langchain langchain-community langchain-chroma unstructured xformers bitsandbytes sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
model_id = "IlyaGusev/saiga_llama3_8b"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm = HuggingFacePipeline(pipeline=query_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [4]:
loader = DirectoryLoader('./documents/', glob="*.md", show_progress=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=100, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(docs[0])
print(all_splits[0])
print(all_splits[1])

  0%|          | 0/5 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
 20%|██        | 1/5 [00:04<00:17,  4.36s/it][nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 5/5 [00:04<00:00,  1.07it/s]

page_content='Решение о создании в СССР интернационального Университета дружбы народов было обусловлено необходимостью оказать помощь странам, завоевавшим свободу от колониальной зависимости на рубеже 1950—1960-х годов. Основной задачей УДН стала подготовка высококвалифицированных национальных кадров для стран Азии, Африки и Латинской Америки, воспитанных в духе дружбы между народами. Кроме того, университет предоставлял возможность получить высшее образование и советской молодёжи.\nПриём студентов осуществлялся через общественные организации и правительственные учреждения, в последующем — через посольства и консульства Советского Союза. Учредителями УДН выступили Всесоюзный центральный совет профессиональных союзов (ВЦСПС), Советский комитет солидарности стран Азии и Африки, Союз советских обществ дружбы и культурных связей с зарубежными странами (ССОД). В 1961 году Университету было присвоено имя Патриса Лумумбы — первого премьер-министра Демократической Республики Конго, одного из с

In [5]:
# Define the path to the pre-trained model you want to use
modelPath = "cointegrated/rubert-tiny2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Год основания?")

print(len(retrieved_docs))
print(retrieved_docs)
print(retrieved_docs[0].page_content)

3
[Document(page_content='Год основания Российского университета дружбы народов: 1960\nПрезидент РУДН: Филиппов Владимир Михайлович\nРектор РУДН: Ястребов Олег Александрович\nКоличество студентов в РУДН: 23 388 (2017): 19 350 очников, 1957 вечерников, 2081 заочников\nКоличество иностранных студентов в РУДН: 5556 (2017 год)\nРасположение РУДН: Москва\nБлижайшая станция метро к РУДН: Университет Дружбы Народов (строится)\nАдрес кампуса РУДН: Москва, улица Миклухо-Маклая, 3—21\nЮридический адрес РУДН: Москва, улица Миклухо-Маклая, 6\nСайт РУДН: rudn.ru\nНаграды РУДН: Орден Дружбы народов, Медаль ЮНЕСКО имени Альберта Эйнштейна', metadata={'source': 'documents/1.md', 'start_index': 512}), Document(page_content='Общежития университета признаны лучшими в 2011 году по итогам конкурса «Наш студенческий дом», проводимого Департаментом семейной и молодёжной политики города Москвы.', metadata={'source': 'documents/5.md', 'start_index': 955}), Document(page_content='В 2010 году университет был наг

In [10]:
import re

rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> Ты ассистент, специализирующийся на вопросах, касающихся Российского университета дружбы народов (РУДН).
    Если вопрос не относится к РУДН, то ты должен вежливо отказать в помощи. Любые вопросы не про Российский университет дружбы народов должны остаться без ответа.
    Используй предложенные фрагменты контекста для формирования ответов. Если ответа на вопрос в контексте нет, скажи, что затрудняешься ответить. Если не уверен в ответе, признай это.
    Ответы должны быть развёрнутыми и лаконичными, но не более трех предложений. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

class LlamaOutputParser:
    def __init__(self, start_substring, end_substring):
        self.start_substring = re.escape(start_substring)
        self.end_substring = re.escape(end_substring)
        self.pattern = re.compile(f"{self.start_substring}.*{self.end_substring}", re.DOTALL)

    def __call__(self, output):
        cleaned_output = re.sub(self.pattern, '', output).strip()
        return cleaned_output

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | LlamaOutputParser("<|begin_of_text|>", "<|end_header_id|>")
)

questions = [
    "Какого цвета бывают кошки?",
    "Кто является ректором Российского университета дружбы народов?",
    "В честь кого назван РУДН?",
    "В честь кого назван российский университет дружбы народов?",
    "Чем собака отличается от кошки?",
    "Что ты можешь рассказать про РУДН?",
    "Что ты можешь рассказать про российский университет дружбы народов?",
    "РУДН это университет или детский сад?",
    "Что ты можешь рассказать про Москву?",
    "Сколько иностранных студентов в РУДН?",
    "Кто ты?"
]

qa_chain.invoke(questions[0])

# for chunk in qa_chain.stream(questions[0]):
#     print(chunk, end="", flush=True)

'Кошки могут иметь разные оттенки меха, включая белый, черный, серый, рыжий, коричневый, оранжевый, желтый, золотистый, тигровой и другие. Цвет кошки зависит от породы и генетических особенностей животного.'

---------------

In [ ]:
!git clone --single-branch --branch dev https://github.com/mklyazhev/rudn_rag.git

Cloning into 'rudn_rag'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 51 (delta 19), reused 44 (delta 12), pack-reused 0
Receiving objects: 100% (51/51), 51.77 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
!pip install -r /content/rudn_rag/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nv

In [ ]:
!python /content/rudn_rag/main.py

/usr/local/lib/python3.10/dist-packages/langchain/embeddings/__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import HuggingFaceEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:548: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import HuggingFacePipeline`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
100% 6/6 [00:03<00:00,  1.79it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume

In [ ]:
# Добавил документы в artifacts/data/raw_data
# Закомментил тело класса Settings в config.py
# Заменил в log_handler.py config.logfile на хардкод /content/rudn_rag/logs/bot_logs.log
# Указал путь к документам в main.py /content/rudn_rag/artifacts/data/raw_data
# Захардкодил токен опенаи в main.py (gpt-3.5-turbo-0125)
# Захардкодил токен бота в main.py